In [3]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import yfinance as yf
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

# Read the data
data_csv = pd.read_csv('btc-2015-2022_case1.csv')
data_csv.dropna(inplace=True)
data_csv.set_index('Date', inplace=True)
data_csv.dropna(inplace=True)

# Split the data into training and test sets
train_df = data_csv[:int(len(data_csv) * 0.8)]
test_df = data_csv[int(len(data_csv) * 0.8):]

# Preprocess the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(train_df.drop(['Adj Close'], axis=1).values)
y_train = scaler.fit_transform(train_df['Adj Close'].values.reshape(-1, 1))
X_test = scaler.fit_transform(test_df.drop(['Adj Close'], axis=1).values)
y_test = scaler.fit_transform(test_df['Adj Close'].values.reshape(-1, 1))

# Reshape the data
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Define the LSTM model function
def create_lstm_model(units=100, dropout_rate=0.2, num_hidden_layers=1):
    model = Sequential()
    for _ in range(num_hidden_layers):
        model.add(Bidirectional(LSTM(units, input_shape=(X_train.shape[1], X_train.shape[2]),
                                     return_sequences=True, activation='tanh')))
        model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(units, activation='tanh')))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
    return model

# Define the hyperparameter grid for grid search
param_grid = {
    'units': [50, 75, 100],
    'dropout_rate': [0.2, 0.25, 0.3],
    'num_hidden_layers': [1, 2, 3],
    'epochs': [100, 500, 1000]
}

# Perform grid search
best_score = float("inf")
best_params = {}
for units in param_grid['units']:
    for dropout_rate in param_grid['dropout_rate']:
        for num_hidden_layers in param_grid['num_hidden_layers']:
            for epochs in param_grid['epochs']:
                model = create_lstm_model(units=units, dropout_rate=dropout_rate, num_hidden_layers=num_hidden_layers)
                model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=0)
                score = model.evaluate(X_train, y_train)[0]
                if score < best_score:
                    best_score = score
                    best_params = {
                        'units': units,
                        'dropout_rate': dropout_rate,
                        'num_hidden_layers': num_hidden_layers,
                        'epochs': epochs
                    }

# Access the best hyperparameters
print("Best Hyperparameters:", best_params)

# Train and evaluate the best model
best_model = create_lstm_model(units=best_params['units'],
                               dropout_rate=best_params['dropout_rate'],
                               num_hidden_layers=best_params['num_hidden_layers'])
best_model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=32)

# Evaluate the model
y_pred_train = best_model.predict(X_train)
y_pred_val = best_model.predict(X_test)

# Perform evaluation metrics
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_val = accuracy_score(y_test, y_pred_val)
precision_train = precision_score(y_train, y_pred_train)
precision_val = precision_score(y_test, y_pred_val)
recall_train = recall_score(y_train, y_pred_train)
recall_val = recall_score(y_test, y_pred_val)
f1_train = f1_score(y_train, y_pred_train)
f1_val = f1_score(y_test, y_pred_val)

# Print evaluation results
print("Training Set:")
print("Accuracy:", accuracy_train)
print("Precision:", precision_train)
print("Recall:", recall_train)
print("F1-Score:", f1_train)

print("Validation Set:")
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1-Score:", f1_val)


71/71 [==============================] - 4s 5ms/step - loss: 2.4258e-05 - mean_absolute_error: 0.0028
